# Import Libraries , Load Functions

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2
import os
import subprocess
import os.path
import time
import math
import csv
#%matplotlib inline
    
from sklearn.datasets import make_regression, make_classification, load_iris
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical,np_utils
from keras.callbacks import EarlyStopping

#Function from Alex Lin 
def bash(string):
    '''run a bash command and return stdout lines as a py list'''
    stdout=(subprocess.Popen(string, shell=True, stdout=subprocess.PIPE).stdout.read())
    output = str(stdout)[2:-1].split('\\n')[:-1]
    #output = str(stdout).split('\\n')[1:-1]
    #output[0] = output[0][2:]
    return output

# Get all the spectrograms in the Data Folder
# then put in a numpy array
# Get labels too

def load_process_data(data_folder, divisor, start_at = "none", stop = "none"):
    lis = bash('ls '+ data_folder)
    
    start_flag = 0
    img_array = []
    labels = []
    
    if start_at == "none":
        start_flag = 1

    for path in lis:
        path_ = data_folder+ path
        
        if path_ == start_at:
            start_flag = 1
        
        if start_flag == 0:
            continue
        
        
        if path_ == stop:
#             img = cv2.imread(path_,1)
#             img_norm = img/255
#             img_norm.resize(int(513/divisor),int(800/divisor),3)
#             img_array.append(img_norm)
#             labels.append(path[0:6])
            break;

        img = cv2.imread(path_,1)
        img_norm = img/255
        img_norm.resize(int(513/divisor),int(800/divisor),3)
        img_array.append(img_norm)
        labels.append(path[0:6])
    
    img_array_nd = np.array(img_array)
    
    le = LabelEncoder()
    y_cat = le.fit_transform(labels)
    y_cat = y_cat.reshape(-1,1)
    
    ohe = OneHotEncoder()
    y_cat = ohe.fit_transform(y_cat).toarray()
    
    X = img_array_nd
    y = y_cat
    
    return X,y,labels

Using TensorFlow backend.


# Process Data

In [4]:
X,y,labels = load_process_data(data_folder="../Data/", divisor = 2,start_at = 'none',stop = '../Data/06_mp3-314.png')

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.1,stratify = y)

In [6]:
X.shape

(2508, 256, 400, 3)

# Create Model

In [7]:
model2 = Sequential()

model2.add(Conv2D(filters = 8,                   # number of filters
                         kernel_size = 5,        # height/width of filter
                         padding = "same",
                         activation='relu',      # activation function 
                         input_shape=(X_test[0].shape))) # shape of input (dimensions of training image)

model2.add(Conv2D(16, kernel_size = 5, activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(Dropout(0.25))
 
model2.add(Conv2D(16, kernel_size = 3, activation='relu'))
model2.add(Conv2D(16, kernel_size = 3, activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))

model2.add(Flatten())
model2.add(Activation('relu'))
# model2.add(Dense(128, activation='relu'))
# model2.add(Dropout(0.5))
model2.add(Dense(6, activation='softmax'))

In [8]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 256, 400, 8)       608       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 252, 396, 16)      3216      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 126, 198, 16)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 126, 198, 16)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 124, 196, 16)      2320      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 122, 194, 16)      2320      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 61, 97, 16)        0         
__________

In [9]:
model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
earlystop = EarlyStopping(monitor='val_acc', min_delta=0.0001, patience=3,verbose=1, mode='auto')
callbacks_list = [earlystop]

# Run Model

In [11]:
model2.fit(X_train, y_train, epochs=100, verbose=1, validation_data=(X_test,y_test),callbacks=callbacks_list)

Train on 2257 samples, validate on 251 samples
Epoch 1/100
2257/2257 [==============================] - 22s 10ms/step - loss: 0.0368 - acc: 0.9911 - val_loss: 0.4418 - val_acc: 0.8845
Epoch 2/100
2257/2257 [==============================] - 22s 10ms/step - loss: 0.0667 - acc: 0.9778 - val_loss: 0.4135 - val_acc: 0.8606
Epoch 3/100
2257/2257 [==============================] - 22s 10ms/step - loss: 0.0746 - acc: 0.9747 - val_loss: 0.4188 - val_acc: 0.8884
Epoch 4/100
2257/2257 [==============================] - 22s 10ms/step - loss: 0.0217 - acc: 0.9929 - val_loss: 0.2662 - val_acc: 0.9124
Epoch 5/100
2257/2257 [==============================] - 22s 10ms/step - loss: 0.0201 - acc: 0.9916 - val_loss: 0.4474 - val_acc: 0.8685
Epoch 6/100
2257/2257 [==============================] - 22s 10ms/step - loss: 0.0660 - acc: 0.9801 - val_loss: 0.3546 - val_acc: 0.8924
Epoch 7/100
2257/2257 [==============================] - 22s 10ms/step - loss: 0.0279 - acc: 0.9907 - val_loss: 0.3825 - val_acc: 0

# Save

In [12]:
model2.save('model_03_6ppl_v2.h5')

# Transfer Learning

Load Model

In [3]:
model = load_model('model_03_6ppl_v2.h5')

In [4]:
model3 = Model(inputs=model.input, outputs=model.get_layer('flatten_1').output)

Load Data for Train , Test

In [5]:
X2,y2,label2 = load_process_data(data_folder = "../Data/", divisor = 2, start_at = '../Data/07_mp3-001.png', stop = '../Data/100_mp-000.png')

Train-Test Split

In [6]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2,y2,test_size = 0.3,stratify = y2)

Initialize SVM

In [7]:
from sklearn import svm
clf = svm.SVC(kernel='rbf', class_weight='balanced')

X_train2_predict = model3.predict(X_train2)
X_test2_predict = model3.predict(X_test2)

y_train2_predict = [np.where(r==1)[0][0] for r in y_train2]
y_test2_predict = [np.where(r==1)[0][0] for r in y_test2]




In [12]:
len(y_train2)
y_train2.shape

(924, 3)

In [9]:
clf.fit(X_train2_predict, y_train2_predict)

SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [10]:
predictions = clf.predict(X_test2_predict)

In [11]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test2_predict, predictions)

0.6262626262626263

In [13]:
confusion_matrix(y_test2_predict, predictions)

array([[128,   0,   7],
       [ 22,   0, 119],
       [  0,   0, 120]])